# Installs

In [1]:
!pip install crewai==0.76.9 crewai_tools==0.13.4 langchain_community==0.3.5

In [2]:
#!pip install CrewAI crewai_tools langchain_community

In [3]:
#!pip install pandas openpyxl

In [4]:
!pip install langchain-groq

In [5]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [6]:
import json
from crewai import Agent, Task, Crew, Process

In [7]:
import os
from google.colab import userdata


# os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
# os.environ["OPENAI_API_KEY"] = "placeholder_key"



from google.colab import userdata
openai_api_key = userdata.get('OPEN_AI_API')
serper_api_key = userdata.get('SERPER_API_KEY')


os.environ["OPENAI_API_KEY"] = userdata.get('OPEN_AI_API')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')


In [8]:
# from dotenv import load_dotenv
# from langchain_groq import ChatGroq

# from crewai import LLM

# llm = LLM(
#     model="groq/llama-3.1-8b-instant"
# )

# Tools

In [9]:
# # Creating tools
!pip install requests  # Install the requests library
import requests

In [10]:
# Patent search tool NEW.

import json
import os
import requests
from crewai_tools import BaseTool

class PatentSearchTool(BaseTool):
    name: str = "Patent Search tool"
    description: str = "Search the internet for Patents."

    # @record_tool(tool_name="Patent search tool")
    def _run(self, query: str) -> str:
        """
        Search the internet for Patents.
        """

        url = "https://google.serper.dev/patents"

        payload = json.dumps({
            "q": query,
            "num": 5,
            "autocorrect": False,
            "tbs": "qdr:d"
        })

        headers = {
            'X-API-KEY': os.getenv('SERPER_API_KEY'),
            'Content-Type': 'application/json'
        }

        response = requests.request("POST", url, headers=headers, data=payload)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the JSON response
            response_data = response.json()

            # Log the entire response structure for debugging
            print("Response data structure:", json.dumps(response_data, indent=2))

            # Ensure 'figures' exists and is a list, and limit to 1 figure
            if 'figures' in response_data and isinstance(response_data['figures'], list):
                if response_data['figures']:  # Check if the list is not empty
                    response_data['figures'] = [response_data['figures'][0]]  # Keep only the first figure
                else:
                    response_data['figures'] = []  # If no figures, set to an empty list
            else:
                response_data['figures'] = []  # If 'figures' key does not exist, set to an empty list

            # Ensure only one URL and thumbnail, assuming they are nested under 'results' (adjust based on actual API response)
            if 'results' in response_data and isinstance(response_data['results'], list) and response_data['results']:
                first_result = response_data['results'][0]  # Get the first result only
                response_data['results'] = [first_result]  # Replace results with only the first result

                # Optionally, if URLs and thumbnails are within each result
                if 'url' in first_result:
                    first_result['url'] = first_result['url']
                if 'thumbnail' in first_result:
                    first_result['thumbnail'] = first_result['thumbnail']

            # Convert the response data to a JSON string and return
            return json.dumps(response_data, indent=2)

        # Handle non-200 status code
        return f"Error: Received status code {response.status_code}"

In [11]:
# search scholar

class ScholarSearchTool(BaseTool):
    name: str = "Scholar search tool"
    description: str = "Search the internet for academic articles."

    # @record_tool(tool_name="Scholar Search Tool")
    def _run(self, query: str) -> str:
        """
        Search the internet for academic articles.
        """

        url = "https://google.serper.dev/scholar"

        payload = json.dumps({
            "q": query,
            "num": 5,
            "autocorrect": False,
            "tbs": "qdr:d"
        })

        headers = {
            'X-API-KEY': os.getenv('SERPER_API_KEY'),
            'Content-Type': 'application/json'
        }

        response = requests.request("POST", url, headers=headers, data=payload)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the JSON response
            response_data = response.json()

        # Convert the news data back to a JSON string
        return json.dumps(response_data, indent=2)

In [12]:
# search news

# class NewsSearchTool(BaseTool):
#     name: str = "News search tool"
#     description: str = "Search the internet for news on the given topic."

#     # @record_tool(tool_name="Scholar Search Tool")
#     def _run(self, query: str) -> str:
#         """
#         Search the internet for news on the given topic.
#         """

#         url = "https://google.serper.dev/news"

#         payload = json.dumps({
#             "q": query,
#             "num": 5,
#             "autocorrect": False,
#             "tbs": "qdr:d"
#         })

#         headers = {
#             'X-API-KEY': os.getenv('SERPER_API_KEY'),
#             'Content-Type': 'application/json'
#         }

#         response = requests.request("POST", url, headers=headers, data=payload)

#         # Check if the request was successful (status code 200)
#         if response.status_code == 200:
#             # Parse the JSON response
#             response_data = response.json()

#         # Convert the news data back to a JSON string
#         return json.dumps(response_data, indent=2)


class NewsSearchTool(BaseTool):
    name: str = "Custom Serper Dev Tool"
    description: str = "Search the internet for news."

    # @record_tool(tool_name="Custom Serper Dev Tool")
    def _run(self, query: str) -> str:
        """
        Search the internet for news.
        """

        url = "https://google.serper.dev/news"

        payload = json.dumps({
            "q": query,
            "num": 10,
            "autocorrect": False,
            "tbs": "qdr:d"
        })

        headers = {
            'X-API-KEY': os.getenv('SERPER_API_KEY'),
            'Content-Type': 'application/json'
        }

        response = requests.request("POST", url, headers=headers, data=payload)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the JSON response
            response_data = response.json()

            # Log the entire response structure for debugging
            print("Response data structure:", json.dumps(response_data, indent=2))

            # Ensure 'news' exists and is a list
            if 'news' in response_data and isinstance(response_data['news'], list):
                if response_data['news']:  # Check if the list is not empty
                    response_data['news'] = [response_data['news'][0]]  # Keep only the first news item
                else:
                    response_data['news'] = []  # If no news items, set to an empty list
            else:
                response_data['news'] = []  # If 'news' key does not exist, set to an empty list

            # Convert the response data to a JSON string
            return json.dumps(response_data, indent=2)

        # Handle non-200 status code
        return f"Error: Received status code {response.status_code}"

In [13]:

# Custom file writer function
def write_to_file(content, directory='project', filename='output.txt'):
    if not os.path.exists(directory):
        os.makedirs(directory)

    file_path = os.path.join(directory, filename)

    # Ensure content is a string before writing
    if not isinstance(content, str):
        content = str(content)

    with open(file_path, 'w') as file:
        file.write(content)

    print(f"Content written to {file_path}")

In [14]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool, \
                         DallETool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool, \
                         VisionTool



In [15]:
#from google.colab import drive
#drive.mount('/content/drive')

In [16]:
#Access Google Drive to write files
#from google.colab import drive
#drive.mount('/content/drive')

In [17]:
#Reading and writing

#read_inputs = DirectoryReadTool(directory='/content/drive/MyDrive/Creative_AI/research_engine/Inputs')
#read_outputs = DirectoryReadTool(directory='/content/drive/MyDrive/Creative_AI/research_engine/Outputs')
#read_baseline= FileReadTool(file_path='/content/drive/MyDrive/Creative_AI/research_engine/Outputs/knowledge_baseline.txt')
#file_read_tool = FileReadTool()

In [18]:
# Direct file reading using Python standard library
#file_path = '/content/drive/MyDrive/Creative_AI/research_engine/Inputs/test.txt'
#with open(file_path, 'r') as file:
#    content = file.read()

#Check if content is being read
#print(content)


In [19]:
#instantiating other tools

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
website_search_tool = WebsiteSearchTool()

#vision_tool = VisionTool(model="dall-e-3",
#                       size="1024x1024",
#                       quality="standard",
#                       n=1)

# AGENTS

In [20]:
manager = Agent(
    role="Project Manager",
    goal="Efficiently manage the research team and ensure the production of world-class research reports",
    backstory=(
        "You are a highly experienced research project manager, overseeing a team dedicated to producing exceptional research reports. "
        "Your responsibilities include ensuring quality at each stage: knowledge collection, analysis, interpretation, and reporting. "
        "Implement multiple revision loops, verifying each stage for accuracy, completeness, and relevance to the topic, purpose, and context."
        "\n\nInstructions:\n"
        "1. **Knowledge Collection**: Begin by collecting all relevant knowledge on the {topic} within the {context}. Gather only reliable and relevant sources.\n"
        "2. **Analysis & Interpretation**: Conduct a detailed analysis, identifying key trends, insights, and implications. Ensure interpretations are accurate and based on the collected data.\n"
        "3. **Quality Checks**: After each stage, evaluate completeness, accuracy, and relevance to {purpose}. If content is lacking, instruct the team to revise it accordingly.\n"
        "4. **Reporting**: Guide the team in drafting a final report. Structure it clearly with an introduction, analysis, and conclusion sections.\n"
        "If any stage is incomplete or fails to meet standards, delegate tasks back to team members for further improvement.\n"
        "Research topic: {topic}\n"
        "Research purpose: {purpose}\n"
        "Research context: {context}\n"
    ),
    # llm=llm,
    allow_delegation=True,
    verbose=True,
    memory=True,
    # max_iter=2,  # Uncomment to limit iterations for efficiency
)


In [21]:
strategist_agent = Agent(
    role="Strategist",
    goal="Analyze and unpack the challenge to reveal its core components and actionable steps",
    backstory=(
        "As an expert strategist, you excel at breaking down complex challenges into clear, manageable elements. "
        "Your goal is to analyze the challenge thoroughly, identifying core issues, key factors, and any underlying assumptions. "
        "From this analysis, outline a clear strategy, highlighting priority areas and actionable steps to address the challenge effectively."
    ),
    instructions=(
        "1. **Identify Core Elements**: Begin by identifying and listing the main components of the challenge. "
        "Break down complex issues into their constituent parts to make them easier to understand.\n"
        "2. **Analyze Key Factors**: Examine each core element, identifying any factors that impact the challenge, such as constraints, risks, and opportunities.\n"
        "3. **Outline Assumptions**: Highlight any assumptions or uncertainties that might influence the approach to solving the challenge.\n"
        "4. **Recommend Actionable Steps**: Conclude by recommending actionable steps based on the analysis, prioritizing actions that are likely to have the highest impact.\n"
    ),
    # llm=llm,
    allow_delegation=False,
    verbose=True,
    memory=True,
    # max_iter=2  # Uncomment to limit the iterations for efficiency
)


In [22]:
researcher_agent = Agent(
    role="Researcher",
    goal="Locate and summarize relevant information to answer questions accurately and comprehensively",
    backstory=(
        "As a diligent and methodical researcher, you are tasked with gathering accessible and highly relevant information "
        "using the tools available. You excel at sifting through large amounts of information, extracting key points, "
        "and presenting them concisely."
        "\n\nInstructions:\n"
        "1. **Identify Relevant Sources**: Focus on finding credible and authoritative sources that address the question directly.\n"
        "2. **Extract Key Points**: From each source, extract only the most relevant details. Avoid overly general information or unrelated data.\n"
        "3. **Summarize Effectively**: Summarize findings in a clear, concise manner. If there are multiple aspects to the question, organize responses by topic.\n"
        "4. **Verify for Completeness**: Ensure that all aspects of the question are addressed comprehensively. If a gap remains, indicate further research areas.\n"
    ),
    # llm=llm,
    allow_delegation=False,
    verbose=True,
    memory=True,
    # max_iter=4  # Uncomment to limit iterations for efficiency
)


In [23]:
scraper_agent = Agent(
    role="Scraper",
    goal="Extract and structure key details from patent documents efficiently",
    backstory=(
        "As an expert scraper, you are skilled at extracting and organizing detailed information from patent documents. "
        "Your role is to retrieve abstracts, claims, technical descriptions, and other relevant information with precision, "
        "ensuring accuracy for subsequent analysis and reporting."
    ),
    instructions=(
        "1. **Identify Key Sections**: Locate and extract the patent's abstract, claims, and technical descriptions. "
        "Pay particular attention to details relevant to the invention's novelty, application, and technical aspects.\n"
        "2. **Structure Data**: Organize extracted information into a structured format, separating sections such as abstract, claims, inventor information, and classifications.\n"
        "3. **Check for Completeness**: Verify that each key section (abstract, claims, and technical descriptions) is captured accurately and is free from omissions.\n"
        "4. **Flag Issues**: If any section is missing or lacks clarity, indicate these issues clearly for further review.\n"
    ),
    # llm=llm,
    allow_delegation=False,
    verbose=True,
    memory=True,
    # max_iter=2  # Uncomment to limit iterations for efficiency
)


In [24]:
writer_agent = Agent(
    role="Writer",
    goal="Summarize technical information from patents into clear, accessible formats for a broad audience",
    backstory=(
        "As a skilled technical writer, you excel at translating complex technical information from patents into concise and "
        "understandable summaries. You focus on key innovations, core claims, and potential applications, ensuring that both technical "
        "and non-technical audiences can grasp the main points effectively."
    ),
    instructions=(
        "1. **Identify Key Innovations**: Begin by summarizing the main innovation of the patent. Focus on what sets it apart and "
        "why it’s significant in the field.\n"
        "2. **Summarize Core Claims**: Highlight the primary claims, explaining the unique aspects of the invention and any technical advantages.\n"
        "3. **Outline Potential Applications**: Describe potential uses and applications of the patent, tailoring language to ensure accessibility for non-technical readers.\n"
        "4. **Use Clear, Concise Language**: Avoid overly technical jargon unless absolutely necessary. Where technical terms are used, provide brief clarifications if they might be unfamiliar to general readers.\n"
        "5. **Maintain Accuracy and Neutral Tone**: Ensure that summaries remain factual and neutral, focusing on the patent content without adding subjective interpretations.\n"
    ),
    # llm=llm,
    allow_delegation=False,
    verbose=True,
    memory=True,
    # max_iter=2  # Uncomment to limit iterations for efficiency
)


In [25]:
insight_agent = Agent(
    role="Insight Strategist",
    goal="Generate innovative, unexpected insights by applying diverse mental models and angles to the problem",
    backstory=(
        "You are an extremely smart insight strategist with a knack for uncovering unique, out-of-the-box perspectives. "
        "While you have expertise in the sector of {sector}, you are proficient in integrating knowledge and methodologies from various disciplines. "
        "You excel at applying creative problem-solving techniques and mental models to generate fresh insights that challenge conventional thinking."
    ),
    instructions=(
        "1. **Analyze from Multiple Angles**: Start by viewing the problem from several distinct perspectives. Apply frameworks from different disciplines (e.g., psychology, economics, history, technology) to uncover novel insights.\n"
        "2. **Use Creative Thinking Techniques**: Leverage creativity tools like lateral thinking, mind mapping, or reverse engineering to challenge assumptions and identify unconventional connections.\n"
        "3. **Look for Unexpected Connections**: Identify patterns or relationships that might not be immediately obvious. Cross-pollinate ideas from unrelated fields to produce original perspectives.\n"
        "4. **Identify Potential Impacts**: For each insight, consider its potential implications. How could it reshape current thinking or lead to unexpected breakthroughs?\n"
        "5. **Contextualize with Sector Knowledge**: Use your sector expertise to ground your insights, but don't be afraid to stretch the boundaries and consider cross-industry trends and innovations.\n"
        "6. **Encourage Provocative Ideas**: Aim to generate insights that challenge the status quo, provoke thought, and inspire new ways of thinking.\n"
        "7. **Keep Insights Actionable**: While creativity is important, ensure that each insight is actionable, offering practical value or guiding strategic decisions.\n"
    ),
    # llm=llm,
    allow_delegation=False,
    verbose=True,
    memory=True,
    # max_iter=2  # Uncomment to limit iterations for efficiency
)


# Tasks

In [26]:
generate_queries_task = Task(
    description=(
        "Based on the provided challenge (e.g., 'biodegradable packaging for food'), generate a set of five related search keywords "
        "that are concise, semantically related, and cover possible variations of the original query. "
        "The queries should be brief and use as few words as possible to maximize relevance and clarity.\n\n"

        "Context:\n"
        "Sector: {sector}\n"
        "Target Clients: {target_clients}\n"
        "Resources: {resources}\n"
        "Strategic Priorities: {strategic_priorities}\n"
        "Project Name: {project_name}\n"
        "Challenge Description: {challenge_description}\n"
        "Purpose: {purpose}\n"
        "Focus Constraints: {focus_constraints}\n\n"

        "Instructions:\n"
        "1. Identify the key concepts in the challenge description.\n"
        "2. Generate search queries that reflect these key concepts while considering variations in phrasing, synonyms, or related terms.\n"
        "3. Ensure that the queries are concise and directly related to the core aspects of the challenge.\n"
        "4. Aim for a balance of broad and specific keywords that would lead to relevant information.\n"
        "5. Ensure that each query is distinct but related, to maximize the range of results without redundancy."
    ),
    expected_output=(
        "A list of five keywords related to the challenge: {challenge_description} that can be used for further searching. "
        "Example: ['biodegradable food packaging', 'compostable packaging materials', 'eco-friendly packaging for food', 'plant-based packaging for perishables']"
    ),
    tools=[],
    agent=strategist_agent
)


In [27]:
patent_search_task = Task(
    description=(
        "For each of the queries generated in Task 1, use the search tool to query Google Patents and retrieve "
        "the top 5 most relevant search results for each query. Ensure that the patents selected are closely related to the original topic, "
        "focusing on titles, abstracts, and URLs. Avoid extracting image links; only text-based information should be extracted. "
        "Each query should be treated separately, ensuring the most relevant patents for each search term are identified."
    ),
    expected_output=(
        "A list of 25 total query results, combining the top 5 relevant patents for each of the 5 queries generated in Task 1. "
        "Each result should include the title, URL, and abstract, formatted in a tabular structure. Do not include images."
    ),
    tools=[PatentSearchTool()],
    agent=researcher_agent
)


In [28]:
scholar_search_task = Task(
    description=(
        "For each of the queries generated in Task 1, use the search tool to query Google Scholar and retrieve "
        "the top 5 most relevant search results for each query. Ensure that the articles selected are closely related to the original topic, "
        "focusing on titles, abstracts, and URLs. Avoid extracting image links or full-text articles; only text-based information should be extracted. "
        "Each query should be treated separately to identify the most relevant academic articles for each search term."
    ),
    expected_output=(
        "A list of 25 total query results, combining the top 5 relevant academic articles for each of the 5 queries generated in Task 1. "
        "Each result should include the title, URL, and abstract, presented in a tabular format. Do not include images or full-text links."
    ),
    tools=[ScholarSearchTool()],
    agent=researcher_agent
)


In [29]:
scrape_content_task = Task(
    description=(
        "For each of the top 5 patent results obtained in Task 2, use the scrape tool to capture the detailed content from the patent pages. "
        "Ensure that the abstract, claims, and any relevant technical information are accurately captured for each patent. "
        "Only extract text-based content—ignore images, diagrams, or any non-textual elements. "
        "The focus should be on key textual information that provides insights into the patent’s claims, technical details, and innovations."
    ),
    expected_output=(
        "A collection of the scraped content for each patent, presented in a structured format. "
        "Each patent should include the abstract, claims, and any important technical details, with the text organized for easy analysis."
    ),
    tools=[scrape_tool],
    agent=scraper_agent
)


In [30]:
select_patents_task = Task(
    description=(
        "Review all the patents identified in the previous task. For each patent, assess its relevance based on the "
        "provided context, including the challenge description, sector, target clients, and strategic priorities. "
        "Use your judgment to determine how closely the patent aligns with the core objectives of the project.\n\n"

        "Context:\n"
        "Sector: {sector}\n"
        "Target Clients: {target_clients}\n"
        "Resources: {resources}\n"
        "Strategic Priorities: {strategic_priorities}\n"
        "Project Name: {project_name}\n"
        "Challenge Description: {challenge_description}\n"
        "Purpose: {purpose}\n"
        "Focus Constraints: {focus_constraints}\n\n"

        "Instructions:\n"
        "1. Review the patent’s title, abstract, claims, and technical content.\n"
        "2. Evaluate how well the patent addresses the key challenges described in the original topic and purpose.\n"
        "3. Rank the patents in order of relevance based on how well they align with the project’s objectives.\n"
        "4. Provide a brief explanation of why each patent was selected, focusing on its relevance to the sector, the challenge, and strategic priorities."
    ),
    expected_output=(
        "A shortlist of the 10 most relevant patents to the original challenge: {challenge_description}. "
        "Present a table with the following columns: patent number, title, awardees, URL, a summary of the abstract, and a brief rationale explaining why each patent is relevant."
    ),
    agent=strategist_agent
)


In [31]:
select_papers_task = Task(
    description=(
        "Review all the research papers identified in the previous task. For each paper, assess its relevance based on the "
        "provided context, including the challenge description, sector, target clients, and strategic priorities. "
        "Use your judgment to determine how closely the paper aligns with the core objectives of the project.\n\n"

        "Context:\n"
        "Sector: {sector}\n"
        "Target Clients: {target_clients}\n"
        "Resources: {resources}\n"
        "Strategic Priorities: {strategic_priorities}\n"
        "Project Name: {project_name}\n"
        "Challenge Description: {challenge_description}\n"
        "Purpose: {purpose}\n"
        "Focus Constraints: {focus_constraints}\n\n"

        "Instructions:\n"
        "1. Review the paper’s title, abstract, and key findings.\n"
        "2. Evaluate how well the paper addresses the key challenges outlined in the original topic and purpose.\n"
        "3. Rank the papers in order of relevance based on how well they align with the project’s objectives.\n"
        "4. Provide a brief explanation of why each paper was selected, focusing on its relevance to the sector, the challenge, and strategic priorities."
    ),
    expected_output=(
        "A shortlist of the 10 most relevant papers to the original challenge: {challenge_description}. "
        "Present a table with the following columns: paper title, authors, URL, summary of the abstract, and a brief rationale explaining why each paper is relevant."
    ),
    agent=strategist_agent
)


In [32]:
summarize_task = Task(
    description=(
        "Based on the selected content from the previous step, first reproduce the output table from the previous task as it is, "
        "with the details of the patents and papers.\n\n"

        "Next, write a concise, clear, and comprehensive summary that explains the content of the selected patents and research papers. "
        "This summary should focus on how these findings relate to the original challenge and purpose. Ensure the summary includes the following:\n"
        "1. **Overview of Innovations**: Highlight the key technological innovations found in the papers and patents.\n"
        "2. **Research Centers and Trends**: Mention where major research is being conducted, which institutions or industries are leading in this space, and any significant geographical concentrations.\n"
        "3. **Research Age and Relevance**: Discuss how recent the research is and whether it's still relevant to the current challenge.\n"
        "4. **Technical Details Translation**: Translate technical content into digestible information for a non-technical audience, ensuring clarity and accessibility.\n"
        "5. **Relevance to Challenge**: Tie each piece of research to the challenge, explaining how it might contribute to solving the problem at hand."
    ),
    expected_output=(
        "Reproduce the output table from the previous task, followed by a clear and concise summary of the patents and papers. "
        "The summary should cover innovations, research trends, the age of research, and its relevance to the challenge."
    ),
    tools=[],  # Placeholder tool for summarizing content
    agent=writer_agent
)


In [33]:
insight_task = Task(
    description=(
        "Generate five novel and thought-provoking insights based on the previous research findings, including the patents and research papers. "
        "An insight should uncover counterintuitive or surprising aspects about the current state of things, often by identifying underlying tensions "
        "or contradictions in the data. For example, there could be a disconnect between what consumers say they want and what is actually available, "
        "or conflicting trends that reveal hidden opportunities. Ensure that each insight is original and provides fresh perspectives.\n\n"

        "Example Insights:\n"
        "- There is a disconnect between the expert recommendation of intuitive eating and people's ingrained habits of following external food rules, "
        "relying on data rather than bodily cues.\n"
        "- There's a mismatch between people’s desire for natural, whole-food nutrition and the prevalence of artificial supplements in the performance nutrition market.\n\n"

        "Instructions:\n"
        "1. **Identify Tensions or Contradictions**: Look for contradictions or tensions in the research that might lead to surprising insights.\n"
        "2. **Generate Catchy Titles**: Each insight should have a compelling and catchy title that summarizes the tension or novel idea.\n"
        "3. **Provide Context**: After each insight, write a brief description that explains the underlying tension and the implications of the finding.\n"
        "4. **Cite Relevant Sources**: Reference the specific papers or patents from previous tasks that support or relate to each insight.\n"
        "5. **Explain Relevance to the Company**: For each insight, explain how it could impact the company and the challenge at hand, and why it is important for their strategic priorities.\n"
        "6. **Link to Challenge and Strategic Priorities**: Ensure that each insight is directly tied to the challenge, sector, target clients, and strategic priorities of the company."
    ),
    expected_output=(
        "A set of five novel, insightful, and cleverly formulated insights that address key contradictions or surprising trends in the research. "
        "Each insight should include a catchy title, a brief description, relevant sources from the research, and an explanation of how it applies to the specific company and its strategic priorities."
    ),
    tools=[],  # Placeholder tool for summarizing content
    context=[summarize_task],  # Link to the summary task for prior research context
    agent=insight_agent
)



# Creating crews

In [34]:
#Patents crew

patents_crew = Crew(
    agents=[
        strategist_agent,
        researcher_agent,
        #scraper_agent,
        writer_agent,

      ],

    tasks=[
        generate_queries_task,
        patent_search_task,
        #patent_search_task,
        #scrape_content_task,
        select_patents_task,
        summarize_task,

    ],

    #process=Process.hierarchical,
    #manager_agent=manager,
    #manager_llm=manager_llm,

    process=Process.sequential,
    #planning=True,
    verbose=True,
	  memory=False,
    #cache=False,
    #share_crew=False,
    #output_log_file="outputs/content_plan_log.txt",
    #max_rpm=50,
    #output_name='patents_output'

)

In [35]:
#Scholar crew

scholar_crew = Crew(
    agents=[
        strategist_agent,
        researcher_agent,
        #scraper_agent,
        writer_agent,
      ],

    tasks=[
        generate_queries_task,
        scholar_search_task,
        #patent_search_task,
        #scrape_content_task,
        select_papers_task,
        summarize_task,

    ],

    #process=Process.hierarchical,
    #manager_agent=manager,
    #manager_llm=manager_llm,

    process=Process.sequential,
    #planning=True,
    verbose=True,
	  memory=False,
    #cache=False,
    #share_crew=False,
    #output_log_file="outputs/content_plan_log.txt",
    #max_rpm=50,
    output_name='scholar_output'

)

In [36]:
#Insights crew

insights_crew = Crew(
    agents=[
        insight_agent,
      ],

    tasks=[
        insight_task,

    ],

    #process=Process.hierarchical,
    #manager_agent=manager,
    #manager_llm=manager_llm,

    process=Process.sequential,
    #planning=True,
    verbose=True,
	  memory=False,
    #cache=False,
    #share_crew=False,
    #output_log_file="outputs/content_plan_log.txt",
    #max_rpm=50,
    output_name='insights'

)

# Inputs

In [37]:
input_1 = {
    # Company inputs
    "company_name": "Coffee Bean Technologies",
    "sector": "Coffee Manufacturing",
    "target_clients": "HoReCa and brands who sell coffee",
    "resources": "Coffee plantations, machinery for harvesting, toasting, brewing, packaging; distribution network",
    "strategic_priorities": "Identifying new products to penetrate existing and new markets, particularly companies targeting younger consumers",

    # Project inputs
    "project_name": "Coffee Bean Coating Technologies",
    "challenge_description": "Identifying technologies that enable coating coffee beans with a thin layer containing color and nutrients",
    "purpose": "Enriching cofee beans by coating them with colors and nutrients ",
    "focus_constraints": " ",

}

input_2 = {
    # Company inputs
    "company_name": "Flipper Eyewear",
    "sector": "Consumer Goods / Eyewear",
    "target_clients": "Individuals seeking versatile and sustainable eyewear solutions",
    "resources": "Innovative eyewear designs with interchangeable lenses and clip-on accessories",
    "strategic_priorities": "Enhance product versatility and user convenience; expand market presence; leverage social media",

    # Project inputs
    "project_name": "Development of Eco-Friendly Materials for Eyewear",
    "challenge_description": "Develop eco-friendly materials for frames and lenses to meet growing consumer demand for sustainable products",
    "purpose": "Align product offerings with environmental sustainability trends to attract eco-conscious consumers and differentiate the brand in the competitive eyewear market",
    "focus_constraints": "Ensure materials meet durability standards while maintaining affordability",
}


input_3 = {
    # Company inputs
    "company_name": "Fastrock Coffee",
    "sector": "Food & Beverage / Coffee",
    "target_clients": "Retailers, foodservice providers, convenience stores, and hospitality industries",
    "resources": "Vertically integrated supply chain; advanced roasting and manufacturing facilities; expertise in sustainable sourcing",
    "strategic_priorities": "Expand global sourcing and supply chain capabilities; invest in sustainable and traceable supply chain practices; enhance product innovation",

    # Project inputs
    "project_name": "Utilization of Coffee By-Products for New Revenue Streams",
    "challenge_description": "Explore alternative uses for coffee by-products to create new revenue streams and reduce waste",
    "purpose": "Develop processes that are cost-effective and environmentally sustainable",
    "focus_constraints": "Ensure new processes are economically viable and align with the company's sustainability goals",
}

# Flow

In [38]:
from crewai.flow.flow import Flow, start, listen
from crewai import Flow
from crewai.flow.flow import listen,start

In [39]:
# class InsightGen(Flow):
#     @start()
#     def find_patents(self):
#         patents_output = patents_crew.kickoff(inputs=inputs)
#         self.state["patents_crew_results"] = patents_output
#         return patents_output

#     @start()
#     def find_scholar(self):
#         scholar_output = scholar_crew.kickoff(inputs=inputs)
#         self.state["scholar_crew_results"] = scholar_output
#         return scholar_output

#     @listen(find_patents)
#     @listen(find_scholar)
#     def generate_insights(self):
#         patents_output = self.state["patents_crew_results"]
#         scholar_output = self.state["scholar_crew_results"]

#         # Combine the outputs into a single dictionary
#         combined_output = {
#             "patents": patents_output,
#             "scholar": scholar_output
#         }

#         # Pass the combined output as a single argument
#         insights = insights_crew.kickoff(combined_output)
#         self.state["insights_crews_results"] = insights
#         return insights


# class InsightGen(Flow):
#     @start()
#     def find_patents(self):
#         patents_output = patents_crew.kickoff(inputs=inputs)
#         self.state["patents_crew_results"] = patents_output
#         return patents_output

#     @start()
#     def find_scholar(self):
#         scholar_output = scholar_crew.kickoff(inputs=inputs)
#         self.state["scholar_crew_results"] = scholar_output
#         return scholar_output

#     @listen(find_patents)
#     @listen(find_scholar)
#     def generate_insights(self):
#         patents_output = self.state["patents_crew_results"]
#         scholar_output = self.state["scholar_crew_results"]

#         # Combine the outputs into a single dictionary
#         combined_output = {
#             "patents": patents_output,
#             "scholar": scholar_output,
#             "sector": inputs["sector"], # Added sector to combined_output
#             "target_clients": inputs["target_clients"],
#             "resources": inputs["resources"],
#             "strategic_priorities": inputs["strategic_priorities"],
#             "project_name": inputs["project_name"],
#             "challenge_description": inputs["challenge_description"],
#             "purpose": inputs["purpose"],
#             "focus_constraints": inputs["focus_constraints"]
#         }

#         # Pass the combined output as a single argument
#         insights = insights_crew.kickoff(combined_output)
#         self.state["insights_crews_results"] = insights
#         return insights

In [40]:
class InsightGen(Flow):
    @start()
    async def start_parallel_execution(self):
        patents_future = patents_crew.kickoff_async(inputs=input_3)
        scholar_future = scholar_crew.kickoff_async(inputs=input_3)

        patents_output, scholar_output = await asyncio.gather(patents_future, scholar_future)

        # Extract raw output from CrewOutput objects
        patents_data = patents_output.raw_output if hasattr(patents_output, 'raw_output') else str(patents_output)
        scholar_data = scholar_output.raw_output if hasattr(scholar_output, 'raw_output') else str(scholar_output)

        self.state["patents_crew_results"] = patents_data
        self.state["scholar_crew_results"] = scholar_data
        return {"patents": patents_data, "scholar": scholar_data}

    @listen(start_parallel_execution)
    def generate_insights(self, parallel_results):
        combined_output = {
            **parallel_results,
            "sector": input_3["sector"],
            "target_clients": input_3["target_clients"],
            "resources": input_3["resources"],
            "strategic_priorities": input_3["strategic_priorities"],
            "project_name": input_3["project_name"],
            "challenge_description": input_3["challenge_description"],
            "purpose": input_3["purpose"],
            "focus_constraints": input_3["focus_constraints"]
        }

        insights = insights_crew.kickoff(combined_output)

        # Extract raw output from CrewOutput
        insights_data = insights.raw_output if hasattr(insights, 'raw_output') else str(insights)

        self.state["insights_crews_results"] = insights_data
        return insights_data

In [41]:
flow = InsightGen()

In [42]:
flow.plot()

from IPython.display import IFrame

IFrame(src='./crewai_flow.html', width='150%', height=600)

Plot saved as crewai_flow.html


# Run Crews

In [43]:
import asyncio

async def run_flow():
  """Helper function to run the flow using the existing loop."""
  return await flow.kickoff_async()

# Get the current event loop or create a new one if it doesn't exist.
loop = asyncio.get_event_loop()

In [44]:
my_insights = await flow.kickoff_async() #should be await flow.kickoff()

# Agent: Strategist
## Task: Based on the provided challenge (e.g., 'biodegradable packaging for food'), generate a set of five related search keywords that are concise, semantically related, and cover possible variations of the original query. The queries should be brief and use as few words as possible to maximize relevance and clarity.

Context:
Sector: Food & Beverage / Coffee
Target Clients: Retailers, foodservice providers, convenience stores, and hospitality industries
Resources: Vertically integrated supply chain; advanced roasting and manufacturing facilities; expertise in sustainable sourcing
Strategic Priorities: Expand global sourcing and supply chain capabilities; invest in sustainable and traceable supply chain practices; enhance product innovation
Project Name: Utilization of Coffee By-Products for New Revenue Streams
Challenge Description: Explore alternative uses for coffee by-products to create new revenue streams and reduce waste
Purpose: Develop processes that are c

In [45]:
# Run crew

# summaries= crew.kickoff(inputs=inputs)

# Save the report to a file
#write_to_file(summaries, directory='/content/drive/MyDrive/Creative_AI/research_engine/Outputs', filename='playground.txt')

In [46]:
# from google.colab import drive
# drive.mount('/content/drive')

In [47]:
my_insights

'### Insight 1: "The Hidden Value of Coffee Pulp: From Tossed Trash to Culinary Treasure"\n**Context**: Despite its abundant nutritional and economic potential, coffee pulp is often discarded, highlighting a contradiction between environmental awareness and practical application. Research indicates that while consumers are increasingly drawn to healthful, whole-food ingredients, coffee pulp—rich in antioxidants—remains underutilized in the food industry. \n**Source**: Utilization of coffee by-products as profitable foods - a mini review. [Link](https://iopscience.iop.org/article/10.1088/1755-1315/672/1/012077/meta)\n**Relevance**: This insight can guide the company in innovating product lines that incorporate coffee pulp as a delicious, nutritious ingredient, aligning with consumer preferences for natural foods and enhancing brand sustainability.\n\n### Insight 2: "Eco-Friendly Fuels: Coffee Grounds as Renewable Energy Sources"\n**Context**: Although the market increasingly calls for s

In [48]:
print(my_insights)

In [49]:
print(type(my_insights))